In [ ]:
def decide_next_step(state: InterviewState) -> InterviewState:
    conversation = state.get("conversation", [])
    current_category = state.get("current_category")

    # 조건처리준비 1: 점수합계
    score_map = {"상": 3, "중": 2, "하": 1}
    total_score = sum(
        score_map.get(turn.get("질문과의 관련성"), 0) +
        score_map.get(turn.get("답변의 구체성"), 0) +
        score_map.get(turn.get("자기이해도"), 0) +
        score_map.get(turn.get("두괄식 답변"), 0)
        for turn in state.get("evaluation", [])
    )

    # 고도화 1: 모든 답변에 대해 답변의 적절성, 질문의 관련성 항목의 점수 합이 총점 96점 이상일 경우 면접을 종료한다.
    if total_score >= 96:
        next_step = "end"
    # 고도화 2: 질문/답변의 횟수가 16회 이상일 경우 면접을 종료한다.
    elif len(conversation) >= 16:
        next_step = "end"
    # 고도화 3: 동일 카테고리에 대한 질문이 2회 이상일 경우 다음 카테고리로 넘어간다.
    else:
        for key_ in state["question_strategy"].keys():
            if state["question_strategy"][key_].get("질문회수") < 2:
                state["current_strategy"] = key_
                next_step = "current_category"
                break

            # 해당사항 없을 경우
            else:
              next_step = "additional_question"

    return {
        **state,
        "next_step": next_step
    }